In [95]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import arff
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import norm
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,  auc, roc_curve
from sklearn.metrics import make_scorer
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.decomposition import PCA
import os

In [96]:
path_to_directory="MDP/"
files = [arff for arff in os.listdir(path_to_directory) if arff.endswith(".arff")]

def toCsv(content): 
    data = False
    header = ""
    newContent = []
    for line in content:
        if not data:
            if "@attribute" in line:
                attri = line.split()
                columnName = attri[attri.index("@attribute")+1]
                header = header + columnName + ","
            elif "@data" in line:
                data = True
                header = header[:-1]
                header += '\n'
                newContent.append(header)
        else:
            newContent.append(line)
    return newContent



In [97]:
# Main loop for reading and writing files
for zzzz,file in enumerate(files):
    with open(path_to_directory+file , "r") as inFile:
        content = inFile.readlines()
        name,ext = os.path.splitext(inFile.name)
        new = toCsv(content)
        with open(name+".csv", "w") as outFile:
            outFile.writelines(new)

In [98]:
cm1 = pd.read_csv('MDP/CM1.csv')
kc3 = pd.read_csv('MDP/KC3.csv')
#kc4 = pd.read_csv('MDP/KC4.csv')
mc2 = pd.read_csv('MDP/MC2.csv')
mw1 = pd.read_csv('MDP/MW1.csv')
pc1 = pd.read_csv('MDP/PC1.csv')
pc2 = pd.read_csv('MDP/PC2.csv')
pc3 = pd.read_csv('MDP/PC3.csv')
pc4 = pd.read_csv('MDP/PC4.csv')
jm1 = pd.read_csv('MDP/JM1.csv')
jm1.rename(columns={"label": "Defective"}, errors="raise", inplace=True)
kc1 = pd.read_csv('MDP/KC1.csv')
mc1 = pd.read_csv('MDP/MC1.csv')
pc5 = pd.read_csv('MDP/PC5.csv')

ar1 = pd.read_csv('SOFTLAB/ar1.csv')
ar3 = pd.read_csv('SOFTLAB/ar3.csv')
ar4 = pd.read_csv('SOFTLAB/ar4.csv')
ar5 = pd.read_csv('SOFTLAB/ar5.csv')
ar6 = pd.read_csv('SOFTLAB/ar6.csv')

eq = pd.read_csv('AEEEM/EQ.csv')
jdt = pd.read_csv('AEEEM/JDT.csv')
lc = pd.read_csv('AEEEM/LC.csv')
ml = pd.read_csv('AEEEM/ML.csv')
pde = pd.read_csv('AEEEM/PDE.csv')


In [99]:
list_nasa = ['cm1','kc3','mc2','mw1','pc1','pc2','pc3','pc4','pc5','jm1','kc1','mc1']
list_data_nasa = [cm1,kc3,mc2,mw1,pc1,pc2,pc3,pc4,pc5,jm1,kc1,mc1]


In [100]:
for dt in list_data_nasa:
    
    dt.loc[dt['Defective'] == 'N', 'Defective'] = 0
    dt.loc[dt['Defective'] == 'Y', 'Defective'] = 1
    dt['Defective'] = dt['Defective'].astype('int64')
    try:
        dt.loc[(dt['DECISION_DENSITY'] == '?'),'DECISION_DENSITY']= -1
        
        change_dtypes = ['DECISION_DENSITY']
        dict_dtypes = {d:'float64' for d in change_dtypes }
        dt = dt.astype(dict_dtypes)
    except:
        pass
    

In [101]:
cm1

,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,...,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL,Defective
0,2,3,0,0,8,4,2,0.22,2,2.00,...,6,0.22,5,10,4,7,9,47.06,9,0
1,3,3,0,2,2,4,2,0.15,2,2.00,...,5,0.11,10,22,5,12,19,26.67,13,0
2,38,35,4,5,70,58,18,0.17,24,2.42,...,51,0.08,150,222,58,32,218,41.90,109,0
3,1,7,5,0,12,12,4,0.10,6,2.00,...,18,0.06,50,79,36,19,68,22.64,41,1
4,9,15,4,14,22,28,8,0.20,14,2.00,...,24,0.11,29,64,19,18,73,57.14,41,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,67,29,10,27,41,56,15,0.13,28,2.00,...,63,0.07,192,274,32,19,228,42.50,119,0
323,9,3,5,0,12,4,2,0.11,2,2.00,...,10,0.05,27,43,21,16,40,40.00,18,0
324,3,3,1,1,2,4,2,0.17,2,2.00,...,6,0.11,11,21,8,14,18,21.43,12,0
325,6,9,3,10,22,16,5,0.16,8,2.00,...,21,0.08,42,72,19,20,61,59.26,32,0


In [102]:
list_relink = ['eq','jdt','lc','ml','pde']
list_data_relink = [eq,jdt,lc,ml,pde]

In [103]:
for dt in list_data_relink:
    dt.drop(index=dt.index[0], axis=0, inplace=True)
    dt['Defective'] = dt['class']
    dt.loc[dt['Defective'] == 'clean', 'Defective'] = 0
    dt.loc[dt['Defective'] == 'buggy', 'Defective'] = 1
    dt.drop('class',axis=1,inplace=True)
    dt['Defective'] = dt['Defective'].astype('int64')

In [104]:
eq.shape

(324, 62)

In [105]:
list_softlab = ['ar1','ar3','ar4','ar5','ar6']
list_data_softlab = [ar1,ar3,ar4,ar5,ar6]

In [106]:
for dt in list_data_softlab:
    dt['Defective'] = dt['defects']
    dt.loc[dt['Defective'] == 'clean', 'Defective'] = 0
    dt.loc[dt['Defective'] == 'buggy', 'Defective'] = 1
    dt.drop('defects', axis=1, inplace=True)
    dt['Defective'] = dt['Defective'].astype('int64')
    

In [107]:
ar1.shape

(121, 30)

In [108]:
import pandas as pd
import numpy as np

def get_top_abs_correlations(df, n=5):
    """
    Computes the absolute correlation matrix of a DataFrame, 
    and returns the column names with the top N highest correlations.

    Args:
        df (pd.DataFrame): The input DataFrame.
        n (int): The number of top correlated columns to return.

    Returns:
        list: A list of column names with the top N highest absolute correlations.
    """
    corr_matrix = df.corr().abs()
    
    # Replace diagonal values with NaN to avoid self-correlation
    corr_matrix = corr_matrix.mask(np.equal(*np.indices(corr_matrix.shape)))
    
    # Stack the correlation matrix to easily find the highest correlations
    corr_series = corr_matrix.stack()
    
    # Get the top N correlation pairs
    top_corr_series = corr_series.nlargest(n)
    
    # Extract column names from the top correlation pairs
    #top_corr_columns = list(set([idx[0] for idx in top_corr_series.index] + 
    #                           [idx[1] for idx in top_corr_series.index]))
    top_corr_columns = list([idx[0] for idx in top_corr_series.index])
    return top_corr_columns


In [109]:
list_dataset = [cm1,kc3]
label_dataset = ['cm1','kc3']

list_test = list_data_nasa + list_data_softlab + list_data_relink
label_test = list_nasa + list_softlab + list_relink

my_index = range(0,len(list_test))
my_dataset = dict(zip(label_test,list_test))

df = list_dataset
#df = pd.concat(df)
len(df)

2

In [110]:
def preprocessing(dataframe, pt = None, scaler = None, show = True):
    
    list_dt = []
    list_dy = []
    
    for dt in dataframe:
        y_dt = dt['Defective']
        X_dt = dt.drop('Defective',axis=1)
        X_dt = X_dt [get_top_abs_correlations(X_dt,30)]
        list_dt.append(X_dt.to_numpy())
        list_dy.append(y_dt)
    
    X = np.concatenate(list_dt, axis=0)
    y = np.concatenate(list_dy, axis=0)
    
    if pt:
        X_norm = pt.transform(X)
    else:
        pt = PowerTransformer()
        X_norm = pt.fit_transform(X)
    
    if scaler:
        X_norm_scale = scaler.transform(X_norm)
    else:
        scaler = MinMaxScaler()
        X_norm_scale = scaler.fit_transform(X_norm)
        
    return pt, scaler, X_norm_scale, y



In [111]:
def heat_map(y,result,title):
    import seaborn as sns
    from sklearn.metrics import confusion_matrix
    sns.set
    _, ax = plt.subplots(figsize=(10,10))
    ax = sns.heatmap(confusion_matrix(y, result), annot=True, fmt='d', cmap= ['#09acec','#089dd9','#078fc5','#0681b1','#05668d','#05368d'], annot_kws={"size": 40, "weight": "bold"})  
    labels = ['No Defect','Defect']
    ax.set_xticklabels(labels, fontsize=30);
    ax.set_yticklabels(labels, fontsize=30,rotation=0);
    ax.set_ylabel('Prediction', fontsize=30,rotation=0);
    ax.set_xlabel('Ground Truth', fontsize=30) #0,1
    ax.set_title(title)
    #plt.close()

In [112]:
def score(y,result):
    accuracy = accuracy_score(y, result)
    f1 = f1_score(y, result, average='weighted')
    precision = precision_score(y, result, average='weighted')
    recall = recall_score(y, result, average='weighted')
    print("Accuracy = " , accuracy, " Precision = ",precision, " Recall = ",recall, " F1-Score = ",f1 )

In [113]:
from sklearn.metrics import make_scorer

scorers = {
            'f1_score': make_scorer(f1_score, average='micro'),
            'precision_score': make_scorer(precision_score, average='micro'),
            'recall_score': make_scorer(recall_score, average='micro'),
            'accuracy_score': make_scorer(accuracy_score)
          }
search_space={}

In [114]:
pt, scaler, X_norm_scale, y = preprocessing(df, show=False)

In [115]:
X_norm_scale.shape, y.shape

((521, 30), (521,))

In [116]:
smote = SMOTE(random_state=42)
X_norm_scale_smote, y_smote = smote.fit_resample(X_norm_scale, y) 
#X_norm_scale_smote, y_smote = X_norm_scale, y    

/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


In [129]:
counter = 0
histories = []

lst_accu_stratified = []
lst_f1_stratified = []
lst_precision_stratified = []
lst_recall_stratified = []

lst_accu_stratified_val = []
lst_f1_stratified_val = []
lst_precision_stratified_val = []
lst_recall_stratified_val = []

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=19)

for train_index, test_index in skf.split(X_norm_scale_smote, y_smote):

    x_train_fold, x_test_fold = X_norm_scale_smote[train_index], X_norm_scale_smote[test_index]
    y_train_fold, y_test_fold = y_smote[train_index], y_smote[test_index]

    
    rte = RandomTreesEmbedding(n_estimators=5000, random_state=0, max_depth=10).fit(x_train_fold)
    x_sparse_embedding_train = rte.transform(x_train_fold)
    x_sparse_embedding_test = rte.transform(x_test_fold)
    
    with open('rte-squezee-{0}.pickle'.format(counter), 'wb') as handle:
        pickle.dump(rte, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    from sklearn.manifold import Isomap
    manifold = PCA(n_components=100)
    x_sparse_manifold_train = manifold.fit_transform(x_sparse_embedding_train)
    x_sparse_manifold_test = manifold.transform(x_sparse_embedding_test)
    
    with open('manifold-squezee-{0}.pickle'.format(counter), 'wb') as handle:
        pickle.dump(manifold, handle, protocol=pickle.HIGHEST_PROTOCOL)
   
    input_X_train1 = layers.Input(
    shape=(x_sparse_manifold_train.shape[1],), name="X_train1")  
    
    input_X_train2 = layers.Input(shape=(x_train_fold.shape[1],), name="X_train2")

    layer_X_train1 = layers.Dense(128,activation='relu',name="X_train1_layer")(input_X_train1)
    layer_X_train1 = layers.BatchNormalization()(layer_X_train1)
    layer_X_train1 = layers.Dropout(0.1)(layer_X_train1)
    layer_X_train1 = layers.Dense(64,activation='relu')(layer_X_train1)
    layer_X_train1 = layers.BatchNormalization()(layer_X_train1)
    layer_X_train1 = layers.Dropout(0.1)(layer_X_train1)

    layer_X_train2 = layers.Dense(128,activation='relu',name="X_train2_layer")(input_X_train2)
    layer_X_train2 = layers.BatchNormalization()(layer_X_train2)
    layer_X_train2 = layers.Dropout(0.1)(layer_X_train2)
    layer_X_train2 = layers.Dense(64,activation='relu')(layer_X_train2)
    layer_X_train2 = layers.BatchNormalization()(layer_X_train2)
    layer_X_train2 = layers.Dropout(0.1)(layer_X_train2)

    # Merge all available features into a single large vector via concatenation
    concat = layers.concatenate([layer_X_train1, layer_X_train2])
    layer_X_train3 = layers.Dense(1024,activation='relu',name="X_train3_layer")(concat)
    layer_X_train3 = layers.BatchNormalization()(layer_X_train3)
    layer_X_train3 = layers.Dropout(0.1)(layer_X_train3)
    layer_X_train3 = layers.Dense(64,activation='relu')(layer_X_train3)
    layer_X_train3 = layers.BatchNormalization()(layer_X_train3)
    layer_X_train3 = layers.Dropout(0.1)(layer_X_train3)

    layer_final = layers.Dense(1, activation='sigmoid', name="classifier")(layer_X_train3)
    sdp_classifier = Model(inputs=[input_X_train1, input_X_train2], outputs=layer_final)

    checkpointer=ModelCheckpoint(filepath='sdp-squezee-{0}.weights.best.keras'.format(counter),verbose=1,save_best_only=True)

    sdp_classifier.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'] )
    
    hist = sdp_classifier.fit([x_sparse_manifold_train,x_train_fold], y_train_fold, 
                    batch_size=32, epochs=100, validation_split= 0.1, callbacks=[checkpointer], 
                    verbose=2, shuffle=True)
    
    with open('sdp-squezee-classifier-{0}.pickle'.format(counter), 'wb') as handle:
        pickle.dump(sdp_classifier, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

    sdp_classifier.load_weights('sdp-squezee-{0}.weights.best.keras'.format(counter))

    histories.append(hist)

    results = sdp_classifier.predict([x_sparse_manifold_train, x_train_fold])
    results = np.round(results)
    
    lst_accu_stratified.append(accuracy_score(results, y_train_fold))
    lst_f1_stratified.append(f1_score(y_train_fold, results, average='weighted') )
    lst_precision_stratified.append(precision_score(results, y_train_fold, average='weighted') )
    lst_recall_stratified.append(recall_score(results, y_train_fold, average='weighted') )
    
    results = sdp_classifier.predict([x_sparse_manifold_test, x_test_fold])
    results = np.round(results)
    
    lst_accu_stratified_val.append(accuracy_score(results, y_test_fold))
    lst_f1_stratified_val.append(f1_score(results, y_test_fold, average='weighted') )
    lst_precision_stratified_val.append(precision_score(results, y_test_fold, average='weighted') )
    lst_recall_stratified_val.append(recall_score(results, y_test_fold, average='weighted') )

    print("Evaluate on training data")
    results = sdp_classifier.evaluate([x_sparse_manifold_train, x_train_fold], y_train_fold)
    print("test loss, test acc:", results)

    print("Evaluate on test data")
    results = sdp_classifier.evaluate([x_sparse_manifold_test, x_test_fold], y_test_fold)
    print("test loss, test acc:", results)
    counter += 1
    

dict_test = {"train_accu":lst_accu_stratified, 
           "train_precision":lst_precision_stratified, "train_recall":lst_recall_stratified, "train_f1":lst_f1_stratified,
           "test_accu":lst_accu_stratified_val, 
           "test_precision":lst_precision_stratified_val, "test_recall":lst_recall_stratified_val, 
           "test_f1":lst_f1_stratified_val,"Input": x_train_fold.shape[1] ,"RTE": x_sparse_manifold_train.shape[1],
            }

dict_test_result = pd.DataFrame.from_dict(dict_test)
dict_test_result.to_csv("hasil.csv")

with open('histories-squezee.pickle'.format(counter), 'wb') as handle:
        pickle.dump(histories, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

Epoch 1/100

Epoch 1: val_loss improved from inf to 0.29592, saving model to sdp-squezee-0.weights.best.keras
23/23 - 4s - loss: 0.7278 - accuracy: 0.6276 - val_loss: 0.2959 - val_accuracy: 1.0000 - 4s/epoch - 196ms/step
Epoch 2/100

Epoch 2: val_loss improved from 0.29592 to 0.22112, saving model to sdp-squezee-0.weights.best.keras
23/23 - 0s - loss: 0.4994 - accuracy: 0.7545 - val_loss: 0.2211 - val_accuracy: 1.0000 - 252ms/epoch - 11ms/step
Epoch 3/100

Epoch 3: val_loss improved from 0.22112 to 0.18780, saving model to sdp-squezee-0.weights.best.keras
23/23 - 0s - loss: 0.4947 - accuracy: 0.7517 - val_loss: 0.1878 - val_accuracy: 1.0000 - 257ms/epoch - 11ms/step
Epoch 4/100

Epoch 4: val_loss did not improve from 0.18780
23/23 - 0s - loss: 0.4349 - accuracy: 0.7992 - val_loss: 0.2053 - val_accuracy: 0.9875 - 147ms/epoch - 6ms/step
Epoch 5/100

Epoch 5: val_loss did not improve from 0.18780
23/23 - 0s - loss: 0.3971 - accuracy: 0.8159 - val_loss: 0.2130 - val_accuracy: 0.9750 - 137m

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


test loss, test acc: [1.0989724397659302, 0.500627338886261]
Evaluate on test data
3/3 [==============================] - 0s 4ms/step - loss: 1.1196 - accuracy: 0.4944
test loss, test acc: [1.1195590496063232, 0.49438202381134033]
Epoch 1/100

Epoch 1: val_loss improved from inf to 0.21962, saving model to sdp-squezee-3.weights.best.keras
23/23 - 3s - loss: 0.7673 - accuracy: 0.6206 - val_loss: 0.2196 - val_accuracy: 0.9625 - 3s/epoch - 148ms/step
Epoch 2/100

Epoch 2: val_loss improved from 0.21962 to 0.19663, saving model to sdp-squezee-3.weights.best.keras
23/23 - 0s - loss: 0.4736 - accuracy: 0.7838 - val_loss: 0.1966 - val_accuracy: 0.9625 - 250ms/epoch - 11ms/step
Epoch 3/100

Epoch 3: val_loss improved from 0.19663 to 0.13686, saving model to sdp-squezee-3.weights.best.keras
23/23 - 0s - loss: 0.4484 - accuracy: 0.7936 - val_loss: 0.1369 - val_accuracy: 0.9750 - 250ms/epoch - 11ms/step
Epoch 4/100

Epoch 4: val_loss did not improve from 0.13686
23/23 - 0s - loss: 0.4464 - accura

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


3/3 [==============================] - 0s 4ms/step - loss: 0.9449 - accuracy: 0.5000
test loss, test acc: [0.9448681473731995, 0.5]
Epoch 1/100

Epoch 1: val_loss improved from inf to 0.25787, saving model to sdp-squezee-7.weights.best.keras
23/23 - 3s - loss: 0.8245 - accuracy: 0.6128 - val_loss: 0.2579 - val_accuracy: 0.9875 - 3s/epoch - 142ms/step
Epoch 2/100

Epoch 2: val_loss did not improve from 0.25787
23/23 - 0s - loss: 0.5101 - accuracy: 0.7437 - val_loss: 0.2810 - val_accuracy: 0.9875 - 145ms/epoch - 6ms/step
Epoch 3/100

Epoch 3: val_loss did not improve from 0.25787
23/23 - 0s - loss: 0.4634 - accuracy: 0.7716 - val_loss: 0.2749 - val_accuracy: 0.9625 - 133ms/epoch - 6ms/step
Epoch 4/100

Epoch 4: val_loss did not improve from 0.25787
23/23 - 0s - loss: 0.4527 - accuracy: 0.7897 - val_loss: 0.3023 - val_accuracy: 0.9125 - 136ms/epoch - 6ms/step
Epoch 5/100

Epoch 5: val_loss did not improve from 0.25787
23/23 - 0s - loss: 0.4032 - accuracy: 0.8217 - val_loss: 0.3532 - val_a

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


test loss, test acc: [0.9690090417861938, 0.5]
Evaluate on test data
3/3 [==============================] - 0s 4ms/step - loss: 0.8801 - accuracy: 0.5000
test loss, test acc: [0.8801222443580627, 0.5]
Epoch 1/100

Epoch 1: val_loss improved from inf to 0.13006, saving model to sdp-squezee-9.weights.best.keras
23/23 - 3s - loss: 0.7263 - accuracy: 0.6421 - val_loss: 0.1301 - val_accuracy: 1.0000 - 3s/epoch - 143ms/step
Epoch 2/100

Epoch 2: val_loss did not improve from 0.13006
23/23 - 0s - loss: 0.5607 - accuracy: 0.7340 - val_loss: 0.1744 - val_accuracy: 0.9875 - 200ms/epoch - 9ms/step
Epoch 3/100

Epoch 3: val_loss did not improve from 0.13006
23/23 - 0s - loss: 0.4933 - accuracy: 0.7716 - val_loss: 0.1644 - val_accuracy: 0.9750 - 132ms/epoch - 6ms/step
Epoch 4/100

Epoch 4: val_loss did not improve from 0.13006
23/23 - 0s - loss: 0.4588 - accuracy: 0.7855 - val_loss: 0.2400 - val_accuracy: 0.9500 - 133ms/epoch - 6ms/step
Epoch 5/100

Epoch 5: val_loss did not improve from 0.13006
23

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


test loss, test acc: [0.9118966460227966, 0.5]


In [130]:
dict_test_result

,train_accu,train_precision,train_recall,train_f1,test_accu,test_precision,test_recall,test_f1,Input,RTE
0,0.529486,0.972809,0.529486,0.395144,0.528090,0.968539,0.528090,0.658319,30,100
1,0.739021,0.834491,0.739021,0.725877,0.674157,0.783793,0.674157,0.692354,30,100
2,0.500627,1.000000,0.500627,0.334031,0.494382,1.000000,0.494382,0.661654,30,100
3,0.570891,0.930140,0.570891,0.477190,0.595506,0.890898,0.595506,0.667392,30,100
4,0.661230,0.873796,0.661230,0.621075,0.651685,0.787294,0.651685,0.677862,30,100
5,0.662484,0.866949,0.662484,0.624178,0.651685,0.866610,0.651685,0.694846,30,100
6,0.501253,0.998750,0.501253,0.336112,0.500000,1.000000,0.500000,0.666667,30,100
7,0.531328,0.947456,0.531328,0.408195,0.511364,0.945506,0.511364,0.646841,30,100
8,0.500000,1.000000,0.500000,0.333333,0.500000,1.000000,0.500000,0.666667,30,100
9,0.507519,0.992594,0.507519,0.349827,0.500000,1.000000,0.500000,0.666667,30,100


In [131]:
best = 1

dbfile = open('rte-squezee-{0}.pickle'.format(best), 'rb')    
rte = pickle.load(dbfile)
dbfile.close()

dbfile = open('manifold-squezee-{0}.pickle'.format(best), 'rb')    
manifold = pickle.load(dbfile)
dbfile.close()

dbfile = open('sdp-squezee-classifier-{0}.pickle'.format(best), 'rb')    
sdp_classifier = pickle.load(dbfile)
dbfile.close()
sdp_classifier.load_weights('sdp-squezee-{0}.weights.best.keras'.format(best))


In [132]:
X_sparse_embedding = rte.transform(X_norm_scale)
X_sparse_manifold = manifold.transform(X_sparse_embedding)
X_norm_scale.shape

(521, 30)

In [133]:
print("Evaluate on training data")
results = sdp_classifier.evaluate([X_sparse_manifold, X_norm_scale], y, batch_size=128)
print("test loss, test acc:", results)

Evaluate on training data
5/5 [==============================] - 0s 3ms/step - loss: 0.7156 - accuracy: 0.5701
test loss, test acc: [0.7156165242195129, 0.5700575709342957]


In [134]:
from sklearn.metrics import classification_report

results = sdp_classifier.predict([X_sparse_manifold,X_norm_scale])
results=np.round(results)
print(classification_report(y,results))

17/17 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.97      0.51      0.67       443
           1       0.24      0.90      0.38        78

    accuracy                           0.57       521
   macro avg       0.61      0.70      0.53       521
weighted avg       0.86      0.57      0.63       521



In [135]:
list_result = []

for label in my_dataset:
    dt = my_dataset[label]
    print(dt.shape, label)
    pt, scaler, X_norm_scale_dt, y_dt = preprocessing([dt],pt, scaler, show=False)
    x_sparse_embedding_dt = rte.transform(X_norm_scale_dt)
    x_sparse_manifold_dt = manifold.transform(x_sparse_embedding_dt)
    
    results = sdp_classifier.predict([x_sparse_manifold_dt,X_norm_scale_dt])
    result  = np.round(results)
    fpr, tpr, thresholds = roc_curve(y_dt, result)
    list_result.append({'label':label, 'accuracy' : accuracy_score(y_dt, result),'precision':precision_score(y_dt, result, average='weighted'),
                  'recall': recall_score(y_dt, result, average='weighted'), 'auc':auc(fpr, tpr),
                        'f1-score':f1_score(y_dt, result, average='weighted')})
    #heat_map(y_dt,result,label)


    

(327, 38) cm1
11/11 [==============================] - 0s 2ms/step
(194, 40) kc3
7/7 [==============================] - 0s 12ms/step
(124, 40) mc2
4/4 [==============================] - 0s 2ms/step
(250, 38) mw1


/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


8/8 [==============================] - 0s 2ms/step
(679, 38) pc1


/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


22/22 [==============================] - 0s 2ms/step
(722, 37) pc2


...

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


59/59 [==============================] - 0s 2ms/step
(1497, 62) pde


/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


47/47 [==============================] - 0s 2ms/step


/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [136]:
df_result = pd.DataFrame(list_result)
df_result

,label,accuracy,precision,recall,auc,f1-score
0,cm1,0.605505,0.880245,0.605505,0.733083,0.668773
1,kc3,0.510309,0.816827,0.510309,0.656470,0.551261
2,mc2,0.645161,0.416233,0.645161,0.500000,0.506009
3,mw1,0.900000,0.810000,0.900000,0.500000,0.852632
4,pc1,0.918999,0.844558,0.918999,0.500000,0.880207
5,pc2,0.977839,0.956170,0.977839,0.500000,0.966883
6,pc3,0.876543,0.768328,0.876543,0.500000,0.818876
7,pc4,0.861417,0.742040,0.861417,0.500000,0.797285
8,pc5,0.729634,0.532366,0.729634,0.500000,0.615582
9,jm1,0.791192,0.625984,0.791192,0.500000,0.698958


In [137]:
df_result[['accuracy','precision','recall','auc','f1-score']].mean()

accuracy     0.809889
precision    0.719711
recall       0.809889
auc          0.517707
f1-score     0.749319
dtype: float64

In [126]:
sdp_classifier.summary()

Model: "model_31"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 X_train1 (InputLayer)       [(None, 100)]                0         []                            
                                                                                                  
 X_train2 (InputLayer)       [(None, 30)]                 0         []                            
                                                                                                  
 X_train1_layer (Dense)      (None, 128)                  12928     ['X_train1[0][0]']            
                                                                                                  
 X_train2_layer (Dense)      (None, 128)                  3968      ['X_train2[0][0]']            
                                                                                           

In [127]:
list_result = []

for label in my_dataset:
    dt = my_dataset[label]
    list_result.append({'Dataset':label, 'Instances':dt.shape[0], 'Features':dt.shape[1],
                        'Defective Instances': dt[dt['Defective'] == 1].shape[0],
                        'Non-Defective Instances': dt[dt['Defective'] == 0 ].shape[0],})


    

In [128]:
df_result = pd.DataFrame(list_result)
df_result

,Dataset,Instances,Features,Defective Instances,Non-Defective Instances
0,cm1,327,38,42,285
1,kc3,194,40,36,158
2,mc2,124,40,44,80
3,mw1,250,38,25,225
4,pc1,679,38,55,624
5,pc2,722,37,16,706
6,pc3,1053,38,130,923
7,pc4,1270,38,176,1094
8,pc5,1694,39,458,1236
9,jm1,7720,22,1612,6108
